# Assignment 10

<font size = "5">
Print your name below

In [7]:
print("angie siaca")

angie siaca


<font size = "5">

Import pandas package

In [8]:
import pandas as pd

<font size = "5">

Import SQL packages

In [9]:
# Write your answer here:


from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text


<font size = "5">

Establish a connection to the SQL server and database ```postgres``` <br>
from Python as done in  class  using <br>
```URL()``` and ```create_engine()```<br>
As we are are in the default database, you can leave out the database argument.

Note: You may have to change some of the login details <br>
depending on your computer

In [10]:
# Write your answer here:

url_server = URL.create(
    "postgresql",
    host = 'localhost',
    username = 'postgres',
    port = 5432,
    password  = '12345')
connection = create_engine(url_server)



<font size = "5">

(a) Upload dataset to SQL

<font size = "3">

- Read the files ```wdi_2018-2023.xlsx``` in the ```data_raw``` folder. <br>
This is a file containing data from the world development indicators for <br>
2018-2023.
- Upload the data to your local SQL server using the <br>
"to_sql" command. Call the new dataset ```wdi_data```
 in the server

In [16]:
# Write your own code
wdi_data  = pd.read_excel("data_raw/wdi_2018-2023.xlsx")

wdi_data.to_sql('wdi_data',
               con = connection,
               if_exists='replace',
               index=False)






578


<font size = "5">

(b) Subset and sort

<font size = "3">

For the following command use ```pd.read_sql()```

- Use ```WHERE``` to get a subset of "wdi_data" in the SQL server to  <br>
which only selects entries related to the USA, <br>
using the column "country_code"
- Chain this with ```ORDER BY year``` to sort in <br>
ascending order <br>

Hints:
- Remember to include a  ```\```  to be able to define strings over multiple lines
- We can include single quotations in the WHERE command <br>
without any additional escape characters

In [ ]:
# Write your answer here:
exampleb = pd.read_sql(text("SELECT * \
                             FROM wdi_data \
                             WHERE country_code = 'USA'\
                             ORDER BY year ASC;"), connection)
print(exampleb.head())





   year   country_name country_code  prop_urban
0  2018  United States          USA      82.256
1  2019  United States          USA      82.459
2  2020  United States          USA      82.664
3  2021  United States          USA      82.873
4  2022  United States          USA      83.084


<font size = "5">

(c) Groupby + Aggregate + Sort

<font size = "3">

For the following command use ```pd.read_sql()```

- From ```wdi_data``` obtain the average of "prop_urban" <br>
by country_name using ```GROUP BY```
- Use ```HAVING ``` to filter countries with an <br>
average proportion of urban population greater than 40%
- Make sure to separate pieces of your logic over seperate lines  by a ```\```

HINT: Use ```HAVING AVG(prop_urban)``` rather than an alias <br>
for the code to work correctly.


In [24]:
# Write your answer here:

avg_urban = pd.read_sql(text( "SELECT country_name, AVG(prop_urban) AS avg_prop_urban  \
                             FROM wdi_data  \
                             GROUP BY country_name  \
                             HAVING AVG(prop_urban) > 0.4  \
                             ORDER BY avg_prop_urban;"), connection)


# inspect the result
print(avg_urban)




                  country_name  avg_prop_urban
0             Papua New Guinea       13.420833
1                      Burundi       13.894167
2                Liechtenstein       14.458500
3                        Niger       16.711167
4                       Rwanda       17.522833
..                         ...             ...
258             Cayman Islands      100.000000
259       Hong Kong SAR, China      100.000000
260  Sint Maarten (Dutch part)      100.000000
261                  Gibraltar      100.000000
262                     Kuwait      100.000000

[263 rows x 2 columns]


<font size = "5">

(d) Groupby + Aggregate

<font size = "3">

(1) Run a query using ```pd.read_sql()```
- From ```wdi_data```, obtain the average, minimum, and maximum <br>
of "prop_urban" by "year" using ```GROUP BY```
- Use ```ORDER BY ``` to sort by the average of "avg_prop_urban"
- Make sure to separate pieces of your logic over seperate lines  by a ```\```
- Store the output as a pandas object called ```wdi_grouped```

(2) Upload
- Reupload the result to the server as ```wdi_grouped_data``` using ```to_sql```

In [35]:
# Write your own code

wdi_grouped = pd.read_sql(text( "SELECT year \
           AVG(prop_urban) AS avg_prop_urban\
           MIN(prop_urban) AS min_prop_urban\
           MAX(prop_urban) AS max_prop_urban\
        FROM wdi_data\
        GROUP BY year\
        ORDER BY avg_prop_urban;"), connection)

# (2) Save the result to a new table in the database
wdi_grouped.to_sql('wdi_grouped',
                   con = connection,
                   if_exists='replace',
                   index=False)
# (3) Inspect the result
wdi_grouped = pd.read_sql(text("SELECT * FROM wdi_grouped;"), connection)
print(wdi_grouped.head())
# (4) Close the connection
connection.close()





ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "("
LINE 1: SELECT year            AVG(prop_urban) AS avg_prop_urban    ...
                                  ^

[SQL: SELECT year            AVG(prop_urban) AS avg_prop_urban           MIN(prop_urban) AS min_prop_urban           MAX(prop_urban) AS max_prop_urban        FROM wdi_data        GROUP BY year        ORDER BY avg_prop_urban;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [41]:
wdi_grouped = pd.read_sql(
    "SELECT year, \
            AVG(prop_urban) AS avg_prop_urban, \
            MIN(prop_urban) AS min_prop_urban, \
            MAX(prop_urban) AS max_prop_urban \
     FROM wdi_data \
     GROUP BY year \
     ORDER BY avg_prop_urban",
    connection
)

print(wdi_grouped)

wdi_grouped.to_sql(
    'wdi_grouped_data',
    con=connection,
    if_exists='replace',
    index=False
)


   year  avg_prop_urban  min_prop_urban  max_prop_urban
0  2018       59.732829          13.032           100.0
1  2019       60.054119          13.250           100.0
2  2020       60.379627          13.345           100.0
3  2021       60.707990          13.456           100.0
4  2022       61.041838          13.582           100.0
5  2023       61.379119          13.723           100.0


6

<font size = "5">

(e) Merge

<font size = "3">

This question depends on your answer to question (d).

For the following command use ```pd.read_sql()```

- From ```wdi_data```, ```LEFT JOIN``` the dataset ```wdi_grouped_data``` <br>
using ```year``` as a merging variable. 
- Hint: See Lecture on SQL for how to merge datasets


In [42]:
# Write your own code
# assuming you already have `connection` open

wdi_merged = pd.read_sql(
    "SELECT d.*, \
            g.avg_prop_urban, \
            g.min_prop_urban, \
            g.max_prop_urban \
     FROM wdi_data AS d \
     LEFT JOIN wdi_grouped_data AS g \
       ON d.year = g.year",
    connection
)

# take a peek
print(wdi_merged.head())




   year                 country_name country_code  prop_urban  avg_prop_urban  \
0  2018                        Aruba          ABW   43.411000       59.732829   
1  2018  Africa Eastern and Southern          AFE   35.893398       59.732829   
2  2018                  Afghanistan          AFG   25.495000       59.732829   
3  2018   Africa Western and Central          AFW   46.709753       59.732829   
4  2018                       Angola          AGO   65.514000       59.732829   

   min_prop_urban  max_prop_urban  
0          13.032           100.0  
1          13.032           100.0  
2          13.032           100.0  
3          13.032           100.0  
4          13.032           100.0  
